In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)
dev = qml.device('default.qubit', wires=3)

In [2]:
def true(phi, theta, omega):
    qml.Rot(phi, theta, omega, wires=0)

def generator(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)
    
def discriminator(w):
    qml.RX(w[0], wires=1)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=1)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=1)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[1,2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [3]:
@qml.qnode(dev)
def true_disc_circuit(phi, theta, omega, disc_weights):
    true(phi, theta, omega)
    discriminator(disc_weights)
    return qml.expval.PauliZ(2)

@qml.qnode(dev)
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval.PauliZ(2)

In [4]:
def prob_real_true(disc_weights):
    true_disc_output = true_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true

def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true # want to minimize this prob

def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights) 
    return cost

def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

In [5]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
gen_weights = np.array([0] + [np.pi] + [0] * 7) #np.random.normal(size=[9])
disc_weights = np.random.normal(size=[9])

opt = qml.GradientDescentOptimizer(0.1)

In [6]:
# train discriminator
for t in range(50):
    disc_weights = opt.step(disc_cost, disc_weights) 
    cost = disc_cost(disc_weights)
    if t % 5 == 0:
        print("Step {}: cost = {}".format(t, cost))

Step 0: cost = -0.137730022663541
Step 5: cost = -0.3367526510285579
Step 10: cost = -0.585151083980837
Step 15: cost = -0.7969873118422433
Step 20: cost = -0.9173028901210573
Step 25: cost = -0.9693490653475476
Step 30: cost = -0.9890722640015006
Step 35: cost = -0.996159686262827
Step 40: cost = -0.998657292396343
Step 45: cost = -0.9995313820751317


In [7]:
 # should be close to one at D's optimum
prob_real_true(disc_weights)

0.9998991972004028

In [8]:
 # should be close to zero at D's optimum
prob_fake_true(gen_weights, disc_weights)

0.00010097970601180561

In [ ]:
# train generator
for t in range(200):
    gen_weights = opt.step(gen_cost, gen_weights)
    cost = -gen_cost(gen_weights)
    if t % 5 == 0:
        print("Step {}: cost = {}".format(t, cost))

Step 0: cost = 0.0001111725580498768
Step 5: cost = 0.0001801160491280429
Step 10: cost = 0.00029239498524308294
Step 15: cost = 0.00047521866636485166
Step 20: cost = 0.000772825807871691
Step 25: cost = 0.0012570467293505083
Step 30: cost = 0.0020442522638955962
Step 35: cost = 0.003322274302820949
Step 40: cost = 0.005392429925702935
Step 45: cost = 0.008733191002601282
Step 50: cost = 0.014091627082812974
Step 55: cost = 0.02260187967872407
Step 60: cost = 0.035906228900611625
Step 65: cost = 0.056196582869428746
Step 70: cost = 0.08599243935071765
Step 75: cost = 0.12738322648944866
Step 80: cost = 0.18062024537591864
Step 85: cost = 0.24265965252654598
Step 90: cost = 0.30715952995007484
Step 95: cost = 0.3668894149216077
Step 100: cost = 0.4170547460917875
Step 105: cost = 0.45687678221658495
Step 110: cost = 0.4888328276617608
Step 115: cost = 0.5170499817642464
Step 120: cost = 0.5461056090964542


In [ ]:
# should be close to one at G's optimum
prob_real_true(disc_weights)

In [ ]:
# hasn't changed from D's turn, putting here for comparison
prob_fake_true(gen_weights, disc_weights)

In [ ]:
 # should be close to zero at joint optimum
disc_cost(disc_weights)